# Assignment For This Week
Develop program menggunakan algoritma k-means clustring untuk data 20newsgroups 3 kelas: Sport, Religion dan Technology. Sumber materi dapat diakses di https://tita.lecturer.pens.ac.id/TextMining_SDT/M10.%20Text%20Clustering/Text%20Clustering%20with%20TF-IDF%20in%20Python.pdf

Dengan ketentuan:

#### A. Data dalam bentuk Bag of Words
1. menggunakan rumus jarak Euclidean Distance
2. menggunakan rumus jarak Cosine Similarity

### B. Data dalam bentuk TF/IDF
1. menggunakan rumus jarak Euclidean Distance
2. menggunakan rumus jarak Cosine Similarity

### C. Data dalam bentuk Count Vectorizer
1. menggunakan rumus jarak Euclidean Distance
2. menggunakan rumus jarak Cosine Similarity

#### D. Data dalam bentuk Vocabulary menggunakan rumus jarak Jaccard Index

Buat analisa hasil klaster menggunakan metric performance DBI dan Silhoute score. 
Sajikan dalma bentuk presentasi. Unggah file ppt nya.